<a href="https://colab.research.google.com/github/danielsaggau/IR_LDC/blob/main/model/SCOTUS/max_pooling_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/danielsaggau/IR_LDC.git

Cloning into 'IR_LDC'...
remote: Enumerating objects: 1415, done.
remote: Counting objects: 100% (411/411), done.
remote: Compressing objects: 100% (271/271), done.
remote: Total 1415 (delta 280), reused 203 (delta 137), pack-reused 1004
Receiving objects: 100% (1415/1415), 3.59 MiB | 12.80 MiB/s, done.
Resolving deltas: 100% (848/848), done.


In [2]:
%cd IR_LDC 

/content/IR_LDC


In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install huggingface_hub
#!python -c "from huggingface_hub.hf_api import HfFolder; HfFolder.save_token('XXXX')"

In [5]:
from torch.utils.data import DataLoader
import math
from sentence_transformers import models, losses
from sentence_transformers import LoggingHandler, SentenceTransformer, InputExample
import logging
from datetime import datetime
import gzip
import sys
import tqdm

In [ ]:
from datasets import load_dataset
dataset = load_dataset("lex_glue", "scotus")

In [7]:
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    TrainingArguments,
    default_data_collator,
    set_seed,
    EarlyStoppingCallback,
    Trainer
)

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
tokenizer = AutoTokenizer.from_pretrained('danielsaggau/scotus_max_pool', use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained('danielsaggau/scotus_max_pool', num_labels=14)

Downloading:   0%|          | 0.00/412 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/222k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/702k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/973 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/167M [00:00<?, ?B/s]

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at danielsaggau/scotus_max_pool and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
tokenized_data = dataset.map(preprocess_function, batched=True)

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
!nvidia-smi

In [ ]:
from transformers import TrainerCallback 
from datasets import load_metric
import numpy as np
import torch as nn

In [ ]:
def compute_metrics(eval_pred):
    metric1 = load_metric("f1")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    micro1 = metric1.compute(predictions=predictions, references=labels, average="micro")["f1"]
    macro1 = metric1.compute(predictions=predictions, references=labels, average="macro")["f1"]
    return { "f1-micro": micro1, "f1-macro": macro1}

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8) # fp16

In [ ]:
!pip install wandb

In [ ]:
from torch import nn
import torch
class LongformerPooler(nn.Module):
    def __init__(self, config, pooling='max'):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.pooling = pooling
        self.activation = nn.Tanh()
        self.max_sentence_length = 512

    def forward(self, hidden_states):
        pooled_output = torch.max(hidden_states, dim=1)[0]
        pooled_output = self.dense(pooled_output)
        pooled_output = self.activation(pooled_output)
        return pooled_output

In [ ]:
model.longformer.pooler = LongformerPooler(model.config)

In [ ]:
model

In [ ]:
!pip install wandb
import wandb
wandb.login()
#fd6f7deb3126d40be9abf77ee753bf45f00e2a9a

In [ ]:
wandb.init(project="scotus")
wandb.init(name="max_linear")

In [ ]:
WANDB_PROJECT='scotus'

In [ ]:
training_args = TrainingArguments(
    output_dir='scotus_max_linear',
    learning_rate=3e-5,
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    push_to_hub=True,
    metric_for_best_model="f1-micro",
    fp16=True,
    report_to="wandb",
    greater_is_better=True,
    lr_scheduler_type='linear',
    run_name="max",
    load_best_model_at_end = True
)

PyTorch: setting up devices


In [ ]:
trainer = Trainer(
    model=model,
    compute_metrics=compute_metrics,
    args=training_args,
    eval_dataset=tokenized_data['test'],
    train_dataset=tokenized_data["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,    
    callbacks = [EarlyStoppingCallback(early_stopping_patience=5)])
trainer.train()

Cloning https://huggingface.co/danielsaggau/scotus_max_cosine into local empty directory.
Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `LongformerForSequenceClassification.forward` and have been ignored: text. If text are not expected by `LongformerForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 5000
  Num Epochs = 10
  Instantaneous batch size per device = 6
  Total train batch size (w. parallel, distributed & accumulation) = 6
  Gradient Accumulation steps = 1
  Total optimization steps = 8340
Automatic Weights & Biases logging enabled, to

Epoch,Training Loss,Validation Loss,F1-micro,F1-macro
1,0.746600,1.072304,0.703571,0.505193
2,0.590200,1.224777,0.710714,0.568672
3,0.438000,1.167673,0.745714,0.623422
4,0.298400,1.450836,0.732857,0.633942
5,0.184900,1.704727,0.735714,0.618457
6,0.110100,1.738184,0.747857,0.630506
7,0.056000,1.922408,0.730714,0.608012
8,0.041800,1.876358,0.743571,0.628393
9,0.035000,1.912344,0.740714,0.624556
10,0.017300,1.920478,0.740714,0.624285


Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on C

Streaming output truncated to the last 5000 lines.
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention

TrainOutput(global_step=8340, training_loss=0.2511282221018839, metrics={'train_runtime': 7955.872, 'train_samples_per_second': 6.285, 'train_steps_per_second': 1.048, 'total_flos': 2.969597381069664e+16, 'train_loss': 0.2511282221018839, 'epoch': 10.0})

In [ ]:
eval_dataset=tokenized_data['validation']
trainer.evaluate(eval_dataset=eval_dataset)

The following columns in the evaluation set don't have a corresponding argument in `LongformerForSequenceClassification.forward` and have been ignored: text. If text are not expected by `LongformerForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1400
  Batch size = 6
Initializing global attention on CLS token...


Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on C

{'eval_loss': 1.4599273204803467,
 'eval_f1-micro': 0.7685714285714286,
 'eval_f1-macro': 0.6814751799563924,
 'eval_runtime': 59.8524,
 'eval_samples_per_second': 23.391,
 'eval_steps_per_second': 3.91,
 'epoch': 10.0}